In [ ]:
using LinearAlgebra
using Plots

using Pkg
Pkg.activate(".")
using SBP_operators

In [ ]:
#=== Problem setup ===#
𝒟x = [0.0,2π]
𝒟y = [0.0,2π]
nx = 41
ny = 41
Dom = Grid2D(𝒟x,𝒟y,nx,ny)

Δt = 0.1* min(Dom.Δx^2,Dom.Δy^2)
t_f = 100Δt


In [ ]:
cx = cy = 0.0
K = 1.0

ũ(x,y,t) = cos(2π*t) * sin(2π*x + cx) * sin(2π*y + cy) #Solution

ũ₀(x,y) = sin(2π*x + cx) * sin(2π*y + cy) #Initial condition

Bxũ(y,t) = cos(2π*t) * sin(cx) * sin(2π*y + cy) #Boundary condition x
Byũ(x,t) = cos(2π*t) * sin(2π*x + cx) * sin(cy) #Boundary condition y

F(x,y,t) = -2π*sin(2π*t)*sin(2π*x+cx)*sin(2π*y+cy) - 
    K * 16π^4 * cos(2π*t)*sin(2π*x+cx)*sin(2π*y+cy) #F = ∂ₜũ - K∇ũ 



In [1]:
nt = 100
u_MMS = zeros(Dom.nx,Dom.ny,nt)
tarry = collect(range(0.0,t_f,step=Δt))
for j = 1:Dom.nx
    for i = 1:Dom.ny
        for k = 1:101
            u_MMS[i,j,k] = ũ(Dom.gridx[i],Dom.gridy[j],tarry[k])
        end
    end
end

surface(Dom.gridy,Dom.gridx,u_MMS[:,:,end])

UndefVarError: UndefVarError: Dom not defined

In [ ]:
# Diffusion coefficients
kx = ky = zeros(Float64,nx,ny) .+ 1.0;

# Boundary conditions from the MMS
BoundaryLeft = Boundary(Dirichlet,Bxũ,Left,1)
BoundaryRight = Boundary(Dirichlet,Bxũ,Right,1)
BoundaryUp = Boundary(Dirichlet,Byũ,Up,2)
BoundaryDown = Boundary(Dirichlet,Byũ,Down,2)

order = 2
method = :cgie

P = VariableCoefficientPDE2D(ũ₀,kx,ky,order,BoundaryLeft,BoundaryRight,BoundaryUp,BoundaryDown)
